<a href="https://colab.research.google.com/github/twyeh/university-physics/blob/main/Rolling_Disc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pydy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 53.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pydy: filename=pydy-0.7.1-py3-none-any.whl size=5227161 sha256=9097615a116b6ca3905445f098729177d303f514325f5781b36a29fdf54d734d
  Stored in directory: /root/.cache/pip/wheels/62/12/5d/1e31d6714ccb545f71b3abc738428c44a6d3b9fcf8efa4794e
Successfully built pydy


In [ ]:
from sympy import symbols, sin, cos, tan
from sympy.physics.mechanics import *
q1, q2, q3, u1, u2, u3  = dynamicsymbols('q1 q2 q3 u1 u2 u3')
q1d, q2d, q3d, u1d, u2d, u3d = dynamicsymbols('q1 q2 q3 u1 u2 u3', 1)
r, m, g = symbols('r m g')
mechanics_printing(pretty_print=False)

In [ ]:
N = ReferenceFrame('N')
Y = N.orientnew('Y', 'Axis', [q1, N.z])
L = Y.orientnew('L', 'Axis', [q2, Y.x])
R = L.orientnew('R', 'Axis', [q3, L.y])
w_R_N_qd = R.ang_vel_in(N)
R.set_ang_vel(N, u1 * L.x + u2 * L.y + u3 * L.z)

In [ ]:
C = Point('C')
C.set_vel(N, 0)
Dmc = C.locatenew('Dmc', r * L.z)
Dmc.v2pt_theory(C, N, R)

r*u2*L.x - r*u1*L.y

In [ ]:
I = inertia(L, m / 4 * r**2, m / 2 * r**2, m / 4 * r**2)
mprint(I)

m*r**2/4*(L.x|L.x) + m*r**2/2*(L.y|L.y) + m*r**2/4*(L.z|L.z)


In [ ]:
kd = [dot(R.ang_vel_in(N) - w_R_N_qd, uv) for uv in L]

In [ ]:
ForceList = [(Dmc, - m * g * Y.z)]
BodyD = RigidBody('BodyD', Dmc, R, m, (I, Dmc))
BodyList = [BodyD]

In [ ]:
KM = KanesMethod(N, q_ind=[q1, q2, q3], u_ind=[u1, u2, u3], kd_eqs=kd)
(fr, frstar) = KM.kanes_equations(BodyList, ForceList)
MM = KM.mass_matrix
forcing = KM.forcing
rhs = MM.inv() * forcing
kdd = KM.kindiffdict()
rhs = rhs.subs(kdd)
rhs.simplify()
mprint(rhs)

Matrix([
[(4*g*sin(q2) + 6*r*u2*u3 - r*u3**2*tan(q2))/(5*r)],
[                                       -2*u1*u3/3],
[                          (-2*u2 + u3*tan(q2))*u1]])
